In [4]:
import nltk
nltk.download('stopwords')

from transformers import AutoModel, AutoConfig, AutoTokenizer

# config = AutoConfig.from_pretrained('microsoft/deberta-v3-large')
# config.max_position_embeddings = 1024
# model = AutoModel.from_pretrained('microsoft/deberta-v3-large', config=config)

checkpoint = 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vierinova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
import torch
import numpy as np
import re
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings("ignore")

In [3]:
def clean_text(text):
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    text = text.replace('-', ',')
    text = text.replace(';', '.')
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')
    text = text.replace('...', '')
    text = text.replace('@', '')

    text = re.sub(r'.\n', ' - ', text)
    text = re.sub(r'--\s?--', '', text)
    
    text = re.sub(r'\s+\?', '?', text)
    text = re.sub(r'\s+,', ',', text)
    
    text = re.sub(r',\s?,', ',', text)
    text = re.sub(r'\s+', ' ', text)


    
    return text.strip().lower()

In [ ]:
dataset = dataset.select()

In [5]:
dataset = dataset.map(lambda x: {'text': clean_text(x['text'])})


Loading cached processed dataset at /home/vierinova/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-992142716dc4055c.arrow


In [6]:
dataset = dataset.filter(lambda x: len(x['text']) > 0 and x['text'][0] != '=', )

Loading cached processed dataset at /home/vierinova/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-cfefd0cd2f2d0810.arrow


In [7]:
train_text = dataset['text']
len(train_text)

859532

In [8]:
train_texts = train_text[:500000]

In [105]:
target2id

{0: 0, -1: -1, 1: 323, 2: 1102, 3: 366, 4: 341, 5: 1084}

In [115]:
test_text, test_target = train_dataset.encoded_texts[:15], train_dataset.targets[:15]

In [116]:
test_text

[1,
 32862,
 3386,
 18258,
 363,
 26520,
 9593,
 8449,
 404,
 261,
 2569,
 38838,
 1504,
 33298,
 287]

In [117]:
test_target

[-1, -1, -1, 0, 0, -1, -1, 0, -1, 3, -1, -1, 0, 0, 0]

In [118]:
tokenizer.decode(test_text)

'senjō no valkyria 3, <unk> chronicles ('

In [5]:
target_ids = tokenizer.encode(".?,-!")[1:-1]
target_token2id = {t: tokenizer.encode(t)[-2] for t in ".?,-!"}
target_ids = list(target_token2id.values())

In [6]:
target_token2id

{'.': 323, '?': 1102, ',': 366, '-': 341, '!': 1084}

In [11]:
# id2target = {
#     0: 0,
#     -1: -1,
# }
# for i, ti in enumerate(target_ids):
#     id2target[ti] = i+1
# target2id = {value: key for key, value in id2target.items()}


In [7]:
#processing texts
id2target = {
    0: 0,
    -1: -1,
}
for i, ti in enumerate(target_ids):
    id2target[ti] = i+1
target2id = {value: key for key, value in id2target.items()}

def create_target(text):
    encoded_words, targets = [], []
    
    words = text.split(' ')

    for word in words:
        target = 0
        for target_token, target_id in target_token2id.items():
            if word.endswith(target_token):
                target = id2target[target_id]

        encoded_word = tokenizer.encode(word, add_special_tokens=False)
        if len(encoded_word) == 0:
            continue
        for w in encoded_word:
            encoded_words.append(w)
        for _ in range(len(encoded_word)-1):
            targets.append(-1)
        targets.append(target)
        
        assert(len(encoded_word)>0)

    encoded_words = [tokenizer.cls_token_id or tokenizer.bos_token_id] +\
                    encoded_words +\
                    [tokenizer.sep_token_id or tokenizer.eos_token_id]
    targets = [-1] + targets + [-1]
    
    return encoded_words, targets


In [13]:
import torch

In [14]:
for k, v in target2id.items():
    if k != -1:
        print(k, tokenizer.decode(v))

0 [PAD]
1 .
2 ?
3 ,
4 -
5 !


In [15]:
# 3-запятая 0-пробел 1-точка 2-вопрос 4-параграф

In [17]:
encoded_texts, targets = [], []
for text in tqdm(train_text[:500_000]):
    enc, tag = create_target(text)
    encoded_texts.append(enc)
    targets.append(tag)

  0%|          | 0/500000 [00:00<?, ?it/s]

In [18]:
train_texts, train_targets, test_texts, test_targets = encoded_texts[:-3000], targets[:-3000], \
                                                        encoded_texts[-3000:], targets[-3000:]

In [19]:
with open('processed_train_words.txt', 'w') as words_file, \
    open('processed_train_targets.txt', 'w') as targets_file:
    for words, targets in zip(train_texts, train_targets):
        words_file.write(' '.join(map(str, words)) + '\n')
        targets_file.write(' '.join(map(str, targets)) + '\n')

In [20]:
with open('processed_val_words.txt', 'w') as words_file, \
    open('processed_val_targets.txt', 'w') as targets_file:
    for words, targets in zip(test_texts, test_targets):
        words_file.write(' '.join(map(str, words)) + '\n')
        targets_file.write(' '.join(map(str, targets)) + '\n')

In [120]:
import os

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoModel, AutoConfig, AutoTokenizer

#creating datasets
class BertDataset(Dataset):
    def __init__(self, path, path_targets, is_train=False):

        self.is_train = is_train
        self.encoded_texts = []
        self.targets = []
        with open(path, 'r') as f:
            for text in f.readlines():
                self.encoded_texts.extend(list(map(int, text.split())))
        with open(path_targets, 'r') as ft:
            for text in ft.readlines():
                self.targets.extend(list(map(int, text.split())))
        
        self.encoded_texts = np.array(self.encoded_texts)
        self.targets = np.array(self.targets)
        idxs = []
        
        for i, (text, target) in enumerate(zip(self.encoded_texts, self.targets)):
            if target >= 1:
                idxs.append(i)
                self.targets[i - 1] = target

        self.encoded_texts = np.delete(self.encoded_texts, idxs)
        self.targets = np.delete(self.targets, idxs)



    def __getitem__(self, idx):
        start_idx = idx * 2048
        start_idx = max(0, start_idx)
        end_idx = start_idx + 2048
        return torch.LongTensor(self.encoded_texts[start_idx: end_idx]),\
               torch.LongTensor(self.targets[start_idx: end_idx])

    def __len__(self):
        return len(self.encoded_texts)//2048 - 1


def collate(batch):
    texts, targets = zip(*batch)
    return torch.stack(texts), torch.stack(targets)

def get_datasets():
    train_dataset = BertDataset('processed_train_words-2.txt', 'processed_train_targets-2.txt', is_train=True)
    valid_dataset = BertDataset('processed_val_words-2.txt', 'processed_val_targets-2.txt')
    return train_dataset, valid_dataset


def get_data_loaders(train_dataset, valid_dataset):
    train_loader = DataLoader(train_dataset, batch_size=2, num_workers=0, collate_fn=collate, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=2, collate_fn=collate)
    return train_loader, valid_loader

In [8]:
from torch import nn

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.x = nn.Sequential(
            nn.Dropout(0,2),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0,2),
            nn.Linear(512, 1))
        self.linear = nn.Linear(768, 6)

    def forward(self, x):
        binary_output = torch.sigmoid(self.x(x))
        x = self.linear(x)
        return x, binary_output

In [9]:
import torch.nn.functional as F

In [10]:
from transformers import AutoModel


class BertPunctuator(nn.Module):
    def __init__(self):
        super().__init__()
        config = AutoConfig.from_pretrained('microsoft/deberta-v3-base')
        config.max_position_embeddings = 2048
        self.base = AutoModel.from_pretrained('microsoft/deberta-v3-base', config=config)

        self.classifier = Classifier()

    def forward(self, x):
        embedding = self.base(x).last_hidden_state
        output, binary_output = self.classifier(embedding)
        output = F.log_softmax(output, dim=-1)
        return output, binary_output

    def train(self, mode=True):
        if mode:
            self.base.train()
            self.classifier.train()
        else:
            self.base.eval()
            self.classifier.eval()
        return self

    def eval(self):
        self.train(False)
        return self


In [124]:
from torch.optim.lr_scheduler import _LRScheduler


class LinearScheduler(_LRScheduler):

    def __init__(self, optimizer, max_steps=10000):
        self.max_steps = max_steps
        self.lr = 0
        super().__init__(optimizer, -1)

    def get_lr(self):
        self.lr = self.base_lrs[-1] * min(1, self._step_count/self.max_steps)
        return [base_lr * min(1, self._step_count/self.max_steps)
                for base_lr in self.base_lrs]

вводим-на самом деле

0-2

2-0

4-4


5-5

In [13]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cuda:0'

In [126]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

train_dataset, valid_dataset = get_datasets()
train_loader, valid_loader = get_data_loaders(train_dataset, valid_dataset)


In [14]:
model = BertPunctuator().to(device)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [143]:
def get_target_weights(targets, output_dim, reduce_empty=True):
    import warnings
    warnings.filterwarnings("ignore")
    weights = compute_class_weight(class_weight='balanced', classes=range(-1, 6), y=targets)[1:] # exclude -1
    return weights


target_weights = torch.Tensor(get_target_weights(train_dataset.targets, 1)).clamp_max(1).to(device)

criterion = nn.NLLLoss(weight=target_weights, reduction='none')

optimizer_args = [
                {'params': model.base.parameters(), 'lr': 3e-5},
                {'params': model.classifier.parameters(), 'lr': 1e-4}
            ]
optimizer = torch.optim.Adam(optimizer_args)
sched = LinearScheduler(optimizer, 500)
num_epochs = 1

In [25]:
# torch.save(model.state_dict(), 'model_state_dict.pt')
# torch.save(optimizer.state_dict(), 'optimizer_state_dict.pt')

In [26]:
# text = text.to(device)
# emb = model.base(text)

In [144]:
for epoch in range(num_epochs):
    model.train()
    torch.save(model.state_dict(), 'model_state_dict_deberta.pt')
    torch.save(optimizer.state_dict(), 'optimizer_state_dict_deberta.pt')
    with tqdm(enumerate(train_loader), total=len(train_loader)) as pbar:
        for i, data in pbar:
            optimizer.zero_grad()
            text, targets = data
            preds, binary_preds = model(text.to(device))

            # Mask some "empty" targets
            mask = ((targets == 0) & (np.random.rand(*targets.shape) < .1)) | (targets > 0)
            mask = mask.to(device)

            # Do not predict output after tokens which are not the end of a word
            not_a_word_mask = (targets == -1).to(device)
            word_mask = ~not_a_word_mask
            targets[not_a_word_mask] = 0

            losses = criterion(preds.reshape(-1, 6), targets.to(device).reshape(-1))
            losses = losses.reshape(text.size(0), text.size(1))
            mask = word_mask * mask
            
            loss = torch.sum(losses * mask) / torch.sum(mask)
            loss.backward()
            
            pbar.set_description(f"loss: {loss.item()}, lr: {optimizer.param_groups[0]['lr']}")

            nn.utils.clip_grad_norm_(model.parameters(), 1.5)
            optimizer.step()
            sched.step()

        # Save model every epoch
        torch.save(model.state_dict(), 'model_state_dict_deberta.pt')
        torch.save(optimizer.state_dict(), 'optimizer_state_dict_deberta.pt')

loss: 0.11346447467803955, lr: 3e-05:  26%|██▌       | 3665/14162 [3:07:35<9:32:28,  3.27s/it]               IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

loss: 0.14867070317268372, lr: 3e-05:  69%|██████▉   | 9795/14162 [7:28:02<1:13:57,  1.02s/it]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

loss: 0.052787523716688156, lr: 3e-05: 100%|██████████| 14162/14162 [8:42:12<00:00,  2.21s/it]


In [145]:
from sklearn.metrics import (classification_report, confusion_matrix,
                f1_score, roc_auc_score, precision_score, recall_score)

def get_classification_report(target, preds):
    report = classification_report(target, preds, output_dict=True)
    report_print = classification_report(target, preds, digits=3)
    return report, report_print


def get_eval_metrics(targets, preds):
    metrics = {}

    preds = np.exp(preds)
    preds = preds.reshape(-1, 6)
    targets = targets.reshape(-1)
    pred_index = preds.argmax(-1)

    cls_report, cls_report_print = get_classification_report(targets, pred_index)
    print(cls_report_print)
    metrics['cls_report'] = cls_report


    macro_precision = precision_score(targets, pred_index, average='macro')
    metrics['precision'] = macro_precision

    macro_recall = recall_score(targets, pred_index, average='macro')
    metrics['recall'] = macro_recall

    macro_f1_score = f1_score(targets, pred_index, average='macro')
    metrics['f_score'] = macro_f1_score

    auc_score = roc_auc_score(targets, preds, average='macro', multi_class='ovo')
    metrics['auc'] = auc_score
    return metrics


In [147]:
# Valid loop

model.eval()
valid_loss = 0
all_valid_preds = []
all_valid_targets = []
for data in tqdm(valid_loader):

    text, targets = data
    with torch.no_grad():
        preds, _ = model(text.to(device))

    word_mask = targets != -1
    preds = preds[word_mask]
    targets = targets[word_mask]

    loss = criterion(preds.view(-1, 6), targets.to(device).view(-1))
    valid_loss += loss.mean().item()
    all_valid_preds.append(preds.detach().cpu().numpy())
    all_valid_targets.append(targets)


valid_loss /= len(valid_loader)
all_valid_preds = np.concatenate(all_valid_preds)
all_valid_targets = np.concatenate(all_valid_targets)

metrics = get_eval_metrics(all_valid_targets, all_valid_preds)
metrics["loss"] = valid_loss

100%|██████████| 79/79 [00:41<00:00,  1.91it/s]


              precision    recall  f1-score   support

           0      0.999     0.921     0.959    251420
           1      0.852     0.961     0.903     13253
           2      0.424     0.800     0.554        35
           3      0.515     0.966     0.671     20072
           4      0.871     0.918     0.894       575
           5      0.349     0.520     0.417       102

    accuracy                          0.926    285457
   macro avg      0.668     0.848     0.733    285457
weighted avg      0.957     0.926     0.935    285457



In [167]:
target2id

{0: 0, -1: -1, 1: 323, 2: 1102, 3: 366, 4: 341, 5: 1084}

In [175]:
tokenizer.decode([341])

'-'

In [148]:
torch.save(model.state_dict(), 'model_state_dict_deberta.pt')

In [17]:
model = BertPunctuator()
model.load_state_dict(torch.load('model_gdown.pt', map_location='cpu'), )

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [18]:
def decode_symb(symb):
    if symb == -1:
        return ''
    if symb == 0:
        return ' '
    return tokenizer.decode(target2id[symb]) + ' '

def predict_model(text):
    encoded_input, targets = create_target(text)
    encoded_input_tens = torch.LongTensor([encoded_input])
    with torch.no_grad():
        output, _ = model(encoded_input_tens)
        
    output = output[0]
    output_idx = torch.argmax(output, dim=-1)
    array = []
    for token, symb, target in zip(encoded_input[1:-1], output_idx.numpy().tolist()[1:-1], targets[1:-1]):
        if target == -1:
            vale = tokenizer.decode(token)
        elif target == 0:
            vale = tokenizer.decode(token) + decode_symb(symb)
        else:
            vale = tokenizer.decode(token) + ' '#  + tokenizer.decode(target2id[target]) + ' '
        array.append(vale)
    return ''.join(array)



In [19]:
print(predict_model('hello how are you'))
print(predict_model('if i could save time in a bottle first thing that i would like to do is to save every day'))
print(predict_model('what does society mean in my opinion it is we people who rule over it'))
print(predict_model('i like apples and tomatos'))
print(predict_model('will i get a possitive mark for this task'))

hello, how are you? 
if i could save time in a bottle, first thing that i would like to do, is to save every day. 
what does society mean? in? my, opinion, it is we, people, who rule over it. 
i like apples and tomatos. 
will? i get a possitive mark for this task? 


In [20]:
print(predict_model('how should i tell varya that i love her'))

how should i tell varya, that i love her? 


In [24]:
text ='I hate him because of his stubbornness How do I handle this'
prepared_text=clean_text(text)

In [25]:
prepared_text

'i hate him because of his stubbornness how do i handle this'

In [27]:
predict_model(prepared_text)

'i hate him, because of his stubbornness. how do i handle this? '

In [31]:
prepared_text = 'a book display with works on critical race theory critical race theory (crt) is a cross disciplinary intellectual and social movement of civil rights scholars and activists who seek to examine the intersection of race society and law in the united states and to challenge mainstream american liberal approaches to racial justice the word critical in its name is an academic term that refers to critical thinking critical theory and scholarly criticism rather than criticizing or blaming people crt is also used in sociology to explain social political and legal structures and power distribution through the lens of race for example the crt conceptual framework is one way to study racial bias in laws and institutions such as the how and why of incarceration rates and how sentencing differs among racial groups in the united states it first arose in the 1970s like other critical schools of thought such as critical legal studies which examines how legal rules protect the status quo a key crt concept is intersectionality the way in which different forms of inequality and identity are affected by interconnections of race class gender and disability scholars of crt view race as a social construct with no biological basis one tenet of crt is that racism and disparate racial outcomes are the result of complex changing and often subtle social and institutional dynamics rather than explicit and intentional prejudices of individuals crt scholars argue that the social and legal construction of race advances the interests of white people at the expense of people of color and that the liberal notion of u.s. law as "neutral" plays a significant role in maintaining a racially unjust social order, where formally color,blind laws continue to have racially discriminatory outcomes'


In [35]:
prepared_text

'a book display with works on critical race theory critical race theory (crt) is a cross disciplinary intellectual and social movement of civil rights scholars and activists who seek to examine the intersection of race society and law in the united states and to challenge mainstream american liberal approaches to racial justice the word critical in its name is an academic term that refers to critical thinking critical theory and scholarly criticism rather than criticizing or blaming people crt is also used in sociology to explain social political and legal structures and power distribution through the lens of race for example the crt conceptual framework is one way to study racial bias in laws and institutions such as the how and why of incarceration rates and how sentencing differs among racial groups in the united states it first arose in the 1970s like other critical schools of thought such as critical legal studies which examines how legal rules protect the status quo a key crt con

In [34]:
predict_model(prepared_text)

'a book, display, with works on critical race theory, critical race theory (crt), is a cross, disciplinary, intellectual, and social movement of civil, rights scholars and activists, who seek to examine the intersection of race, society, and law in the united states, and to challenge mainstream american, liberal approaches to racial justice. the word critical in its name, is an academic term that refers to critical, thinking, critical theory, and scholarly criticism, rather than criticizing or blaming people. crt is also used in sociology, to explain social, political, and legal structures, and power distribution, through the lens of race. for example, the crt conceptual framework is one way to study racial bias in laws and institutions, such as the how and why of incarceration rates, and how sentencing differs among racial groups in the united states. it first arose in the 1970s, like other critical schools of thought, such as critical legal studies, which examines how legal rules pro

In [33]:
predict_model(prepared_text)

'a book, display, with works on critical race theory, critical race theory (crt), is a cross, disciplinary, intellectual, and social movement of civil, rights scholars and activists, who seek to examine the intersection of race, society, and law in the united states, and to challenge mainstream american, liberal approaches to racial justice. the word critical in its name, is an academic term that refers to critical, thinking, critical theory, and scholarly criticism, rather than criticizing or blaming people. crt is also used in sociology, to explain social, political, and legal structures, and power distribution, through the lens of race. for example, the crt conceptual framework is one way to study racial bias in laws and institutions, such as the how and why of incarceration rates, and how sentencing differs among racial groups in the united states. it first arose in the 1970s, like other critical schools of thought, such as critical legal studies, which examines how legal rules pro

In [22]:
prepared_text

'a book display with works on critical race theory critical race theory (crt) is a cross,disciplinary intellectual and social movement of civil,rights scholars and activists who seek to examine the intersection of race, society, and law in the united states and to challenge mainstream american liberal approaches to racial justice. the word critical in its name is an academic term that refers to critical thinking, critical theory, and scholarly criticism, rather than criticizing or blaming people. crt is also used in sociology to explain social, political, and legal structures and power distribution through the lens of race. for example, the crt conceptual framework is one way to study racial bias in laws and institutions, such as the how and why of incarceration rates and how sentencing differs among racial groups in the united states. it first arose in the 1970s, like other critical schools of thought, such as critical legal studies, which examines how legal rules protect the status q

In [23]:
predict_model(prepared_text)

'a book, display, with works on critical race theory, critical race theory (crt), is a cross,disciplinary, intellectual, and social movement of civil,rights, scholars and activists, who seek to examine the intersection of race, society, and law in the united states, and to challenge mainstream american, liberal approaches to racial justice. the word critical in its name, is an academic term that refers to critical thinking, critical theory, and scholarly criticism, rather than criticizing or blaming people. crt is also used in sociology, to explain social, political, and legal structures, and power distribution, through the lens of race. for example, the crt conceptual framework is one way to study racial bias in laws and institutions, such as the how and why of incarceration rates, and how sentencing differs among racial groups in the united states. it first arose in the 1970s, like other critical schools of thought, such as critical legal studies, which examines how legal rules prote

In [29]:
# Valid loop

model.eval()
valid_loss = 0
all_valid_preds = []
all_valid_targets = []
for data in tqdm(valid_loader):

    text, targets = data
    with torch.no_grad():
        preds, _ = model(text.to(device))

    word_mask = targets != -1
    preds = preds[word_mask]
    targets = targets[word_mask]

    loss = criterion(preds.view(-1, 4), targets.to(device).view(-1))
    valid_loss += loss.mean().item()
    all_valid_preds.append(preds.detach().cpu().numpy())
    all_valid_targets.append(targets)


valid_loss /= len(valid_loader)
all_valid_preds = np.concatenate(all_valid_preds)
all_valid_targets = np.concatenate(all_valid_targets)

metrics = get_eval_metrics(all_valid_targets, all_valid_preds)
metrics["loss"] = valid_loss

100% 197/197 [00:08<00:00, 21.89it/s]


              precision    recall  f1-score   support

           0      0.998     0.920     0.957    266444
           1      0.770     0.969     0.858       614
           2      0.280     0.857     0.423        35
           3      0.483     0.972     0.646     20210

    accuracy                          0.924    287303
   macro avg      0.633     0.930     0.721    287303
weighted avg      0.961     0.924     0.935    287303



In [30]:
len(metrics

{'cls_report': {'0': {'precision': 0.9977659405634387,
   'recall': 0.9202383990632178,
   'f1-score': 0.9574353016976288,
   'support': 266444},
  '1': {'precision': 0.7697283311772316,
   'recall': 0.9690553745928339,
   'f1-score': 0.8579668348954579,
   'support': 614},
  '2': {'precision': 0.2803738317757009,
   'recall': 0.8571428571428571,
   'f1-score': 0.4225352112676056,
   'support': 35},
  '3': {'precision': 0.4831129246349737,
   'recall': 0.9724888668975754,
   'f1-score': 0.6455363594560862,
   'support': 20210},
  'accuracy': 0.9240105393956902,
  'macro avg': {'precision': 0.6327452570378362,
   'recall': 0.929731374424121,
   'f1-score': 0.7208684268291946,
   'support': 287303},
  'weighted avg': {'precision': 0.960988526934326,
   'recall': 0.9240105393956902,
   'f1-score': 0.9352173897214812,
   'support': 287303}},
 'precision': 0.6327452570378362,
 'recall': 0.929731374424121,
 'f_score': 0.7208684268291946,
 'auc': 0.9930891773177869,
 'loss': 0.069513784352746

In [86]:
len(train_dataset.targets)

64499571

In [87]:
len(valid_dataset.targets)

363140